In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser #take the chat message and convert it into the string

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

C:\Users\HP\AppData\Local\Temp\ipykernel_6444\2159996570.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic":"Ananay"})

'Why did Ananay bring a ladder to the bar? \nBecause he heard the drinks were on the house!'

More complex chain
And Runnable Map to supply user-provided inputs to the prompt.

Normal RAG

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [7]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

C:\Users\HP\AppData\Local\Temp\ipykernel_6444\1824015626.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings()
d:\Functions, Tools and Agents\functionenv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
retriever.get_relevant_documents("where did harrisonn work")

C:\Users\HP\AppData\Local\Temp\ipykernel_6444\208383055.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrisonn work")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [9]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

Retreival Augmented Generation Pipeline

In [10]:
template = """Answer the question based on the following context:{context}
Question:{question}"""

prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain.schema.runnable import RunnableMap # parallel execution of the multiple functions

In [12]:
chain = RunnableMap({
    "context": lambda x : retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [13]:
chain.invoke({"question":"where did the harrison worked at ?"})

'Harrison worked at Kensho.'

Bind and OpenAI functions

In [14]:
functions = [{
    "name":"weather_search",
    "description":"Search for weather given an airport code",
    "parameters":{
        "type":"object",
        "properties":{
            "airport_code":{
                "type":"string",
                "description":"The airport code we need the weather for"
            },
        },
        "required":["airport_code"]
    }
}]

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}")
    ]
)

model = ChatOpenAI(temperature=0).bind(functions=functions)

In [16]:
runnable = prompt | model

In [17]:
runnable.invoke({"input":"What is the weather like in DL"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"DL"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 65, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-75e3fa07-fea2-444a-9869-e859b81b7f55-0')

We can have the multiple functions

In [18]:
[
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code we need the weather for"
                }
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "To get the sports news of the given team name",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The team name we need the sports news for"
                }
            },
            "required": ["team_name"]
        }
    }
]


[{'name': 'weather_search',
  'description': 'Search for weather given an airport code',
  'parameters': {'type': 'object',
   'properties': {'airport_code': {'type': 'string',
     'description': 'The airport code we need the weather for'}},
   'required': ['airport_code']}},
 {'name': 'sports_search',
  'description': 'To get the sports news of the given team name',
  'parameters': {'type': 'object',
   'properties': {'team_name': {'type': 'string',
     'description': 'The team name we need the sports news for'}},
   'required': ['team_name']}}]

In [19]:
model = model.bind(functions = functions)

In [20]:
runnable = prompt | model

In [21]:
runnable.invoke({"input":"how did the patriots do yesterday"})

AIMessage(content="I'm not sure which Patriots team you are referring to. Could you please provide more context or specify the sport or league?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 65, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a14f4dba-1027-4467-9fea-e0ebbb1efe62-0')

FallBacks- mechanism or alternative behavior that is used when the primary option or functionality fails or is unavailable.

In [22]:
from langchain.llms import OpenAI
import json

In [23]:
simple_model = OpenAI(
    temperature = 0,
    max_tokens = 1000,
    model = "text-davinci-001"
)
simple_chain = simple_model | json.loads

C:\Users\HP\AppData\Local\Temp\ipykernel_6444\266987788.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(


In [24]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

Interface

In [25]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser #take the chat message and convert it into the string

In [26]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about the {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

runnable = prompt | model | output_parser

In [27]:
# This is on the single input
runnable.invoke({"topic":"Laptop"})

'Why did the laptop go to therapy? \n\nIt had too many unresolved issues with its mouse!'

In [28]:
# if we want to do it on the batch of inputs
runnable.batch([{"topic":"Table"},{"topic":"bear"}])

['Why did the table wear glasses? \n\nBecause it had too many legs to see properly!',
 'Why did the bear break up with his girlfriend? \n\nBecause she was unbearable!']